<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Code_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np

In [12]:
max_seq_length = 150

In [13]:
ratings1 = pd.read_csv('/content/drive/My Drive/Movielensdata/ml25m/ratings.csv')
ratings = ratings1.iloc[1:1000,:]
ratings.head()

,userId,movieId,rating,timestamp
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
5,1,1088,4.0,1147868495
6,1,1175,3.5,1147868826
7,1,1217,3.5,1147878326
8,1,1237,5.0,1147868839
9,1,1250,4.0,1147868414
10,1,1260,3.5,1147877857


In [14]:
ratings.sort_values(by=['userId','timestamp'],inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
ratings.head(10)

,userId,movieId,rating,timestamp
36,1,5952,4.0,1147868053
13,1,2012,2.5,1147868068
12,1,2011,2.5,1147868079
11,1,1653,4.0,1147868097
9,1,1250,4.0,1147868414
...,...,...,...,...
65,1,27193,3.0,1147879774
32,1,5684,2.0,1147879797
45,1,7318,2.0,1147879850
48,1,7361,5.0,1147880055


In [16]:
users_total = len(ratings['userId'].unique())
print(users_total)

4


In [17]:
movie_list = []
for i in range(users_total):
  list1 = []
  list1 = ratings.loc[ratings['userId'] ==(i+1),['movieId']]['movieId'].tolist()
  if len(list1)>max_seq_length:
    list1 = list1[(len(list1)-max_seq_length):]
  elif len(list1)<max_seq_length:
    for j in range((max_seq_length-len(list1))):
      list1.append(0)
  movie_list.append(list1)
print(movie_list)

[[5952, 2012, 2011, 1653, 1250, 6539, 6377, 3448, 1088, 899, 4308, 2161, 6711, 3949, 8360, 5878, 306, 1175, 307, 1237, 7327, 8154, 7234, 2843, 4144, 7365, 2068, 4422, 4973, 6016, 8873, 2692, 27721, 7323, 6954, 8014, 7939, 6370, 8973, 4703, 31956, 5147, 8786, 1260, 2351, 7940, 7209, 8685, 7820, 7937, 7938, 8405, 4325, 2632, 1217, 8729, 5912, 5767, 665, 2573, 27266, 8327, 32591, 5269, 3569, 27193, 5684, 7318, 7361, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [733, 266, 333, 8961, 5816, 6157, 4857, 4571, 4299, 3510, 3479, 3916, 3107, 2496, 2694, 2720, 2643, 1957, 2273, 1587, 1488, 1431, 1283, 1299, 34162, 27821, 7007, 7004, 6753, 6156, 6213, 6287, 5574, 3889, 110, 1610, 1291, 4019, 40815, 7143, 457, 6333, 7386, 3114, 356, 33794, 5418, 4896, 5349, 1210, 4306, 8368, 8360, 3793, 4995, 4963, 8636

In [18]:
rating_list =[]
for i in range(users_total):
  list2 = []
  list2 = ratings.loc[ratings['userId'] ==(i+1),['rating']]['rating'].tolist()
  if len(list2)>max_seq_length:
    list2 = list2[(len(list2)-max_seq_length):]
  elif len(list2)<max_seq_length:
    for j in range((max_seq_length-len(list2))):
      list2.append(0)
  rating_list.append(list2)
print(rating_list)

[[4.0, 2.5, 2.5, 4.0, 4.0, 3.5, 4.0, 4.0, 4.0, 3.5, 3.0, 3.5, 5.0, 5.0, 4.0, 4.0, 3.5, 3.5, 5.0, 5.0, 3.5, 5.0, 4.5, 4.5, 5.0, 4.0, 2.5, 3.0, 4.5, 5.0, 3.0, 5.0, 3.0, 3.5, 3.5, 3.5, 2.5, 4.5, 4.0, 4.0, 3.5, 4.0, 4.0, 3.5, 4.5, 4.5, 4.0, 1.0, 2.5, 3.0, 2.5, 3.5, 5.0, 5.0, 3.5, 3.5, 3.0, 5.0, 5.0, 4.0, 4.5, 5.0, 5.0, 0.5, 5.0, 3.0, 2.0, 2.0, 5.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4.5, 1.0, 5.0, 4.5, 5.0, 3.5, 1.0, 1.5, 4.5, 4.0, 2.0, 3.0, 3.0, 4.0, 4.0, 0.5, 0.5, 3.5, 2.0, 1.0, 5.0, 3.0, 4.0, 1.0, 0.5, 4.5, 0.5, 2.0, 3.0, 2.0, 2.5, 2.0, 3.0, 2.5, 5.0, 5.0, 5.0, 3.5, 5.0, 5.0, 5.0, 5.0, 4.0, 4.5, 4.5, 5.0, 5.0, 5.0, 5.0, 5.0, 4.5, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 4.5, 4.0, 5.0, 4.0, 4.5, 5.0, 4.0, 4.0, 5.0, 3.0, 4.0, 3.5, 4.0, 4.5, 5.0, 4.5, 3.5, 5.0, 4.5, 4.5, 5.0, 4.0, 3.0, 3.0, 4.0

In [19]:
movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(users_total)])
movies_transpose.index.names = ['userId']
print(movies_transpose)

         0      1     2       3       4    ...    145     146    147   148   149
userId                                     ...                                  
1       5952   2012  2011    1653    1250  ...      0       0      0     0     0
2        733    266   333    8961    5816  ...   2987    1293   7090  1136  2150
3       4383  52950  2021  175197  182823  ...  88125  106489  81834  4262  1732
4       8665   8961  4993    1580    5444  ...      0       0      0     0     0

[4 rows x 150 columns]


In [20]:
ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(users_total)])
ratings_transpose.index.names = ['userId']
print(ratings_transpose)

        0    1    2    3    4    5    6    ...  143  144  145  146  147  148  149
userId                                     ...                                   
1       4.0  2.5  2.5  4.0  4.0  3.5  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2       4.5  1.0  5.0  4.5  5.0  3.5  1.0  ...  0.5  3.0  0.5  5.0  2.0  5.0  5.0
3       4.0  4.5  4.0  3.5  3.0  4.5  4.0  ...  2.5  4.0  3.5  3.5  3.5  4.5  4.5
4       3.5  5.0  4.5  4.5  4.0  4.0  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[4 rows x 150 columns]
